In [1]:
from Preprocessing.MECO_data_split import concat_MECO_langs, split_into_time_series, lang_list
import numpy as np

In [33]:
data = concat_MECO_langs(lang_list)

In [41]:
X, X_test, y, y_test, demo, demo_test = split_into_time_series(data, length=180)

In [35]:
X_flat = np.apply_along_axis(lambda x: x.flatten(order='C'), 0, X)
X_flat

array([[[1433,  167,  297],
        [1470,  167,  140],
        [1578,  171,  196],
        ...,
        [1327,  222,  264],
        [1456,  235,  174],
        [1534,  231,  154]],

       [[ 879,   55,  225],
        [ 205,   49,  660],
        [ 941,  348,  430],
        ...,
        [ 794,  252,  255],
        [1446,  260,  308],
        [ 656,   56,  201]],

       [[ 114,   59,  116],
        [ 226,  117,  144],
        [ 185,  125,  169],
        ...,
        [1202,  139,  241],
        [1276,  142,  229],
        [1382,  141,  295]],

       ...,

       [[ 128,  117,  208],
        [ 192,  102,  376],
        [ 324,  107,  146],
        ...,
        [1415,  130,  215],
        [ 163,  171,  196],
        [ 154,  184,  149]],

       [[ 306,  518,  107],
        [ 440,  557,  174],
        [ 552,  547,  176],
        ...,
        [1095,  585,  185],
        [1264,  578,  180],
        [1215,  567,  149]],

       [[ 218,  408,  178],
        [ 175,  147,  196],
        [ 139,  

In [42]:
X = (X - np.mean(X)) / np.std(X)

X_dur = X.T[2].T
X_test_dur = X_test.T[2].T

In [43]:
from keras import Sequential, Model
from keras.layers import Input, LSTM, RepeatVector, Dense, TimeDistributed, Conv1D, MaxPooling1D, Conv1DTranspose

import tensorflow as tf
# tf.config.set_visible_devices([], 'GPU')

In [38]:
class DtwLoss(tf.keras.losses.Loss):
    def __init__(self, batch_size: int = 32):
        super(DtwLoss, self).__init__()
        self.batch_size = batch_size

    def call(self, y_true, y_pred):
        tmp = []
        for item in range(self.batch_size):
            tf.print(f'Working on batch: {item}\n')
            s = y_true[item, :]
            t = y_pred[item, :]
            n, m = len(s), len(t)
            dtw_matrix = []
            for i in range(n + 1):
                line = []
                for j in range(m + 1):
                    if i == 0 and j == 0:
                        line.append(0)
                    else:
                        line.append(np.inf)
                dtw_matrix.append(line)

            for i in range(1, n + 1):
                for j in range(1, m + 1):
                    cost = tf.abs(s[i - 1] - t[j - 1])
                    last_min = tf.reduce_min([dtw_matrix[i - 1][j], dtw_matrix[i][j - 1], dtw_matrix[i - 1][j - 1]])
                    dtw_matrix[i][j] = tf.cast(cost, dtype=tf.float32) + tf.cast(last_min, dtype=tf.float32)

            temp = []
            for i in range(len(dtw_matrix)):
                temp.append(tf.stack(dtw_matrix[i]))

            tmp.append(tf.stack(temp)[n, m])
        return tf.reduce_mean(tmp)

In [46]:
input_img = Input(shape=(180,))
encoder = Dense(128, activation='relu')(input_img)
encoder = Dense(64, activation='relu')(encoder)

encoder = Dense(32, activation='relu')(encoder)

decoder = Dense(64, activation='relu')(encoder)
decoder = Dense(128, activation='relu')(decoder)
output_img = Dense(180, activation='relu')(decoder)

autoencoder = Model(input_img, output_img)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

autoencoder.fit(X_dur, np.float32(X_dur), epochs=10)

Epoch 1/10
72/72 [==============================] - 1s 8ms/step - loss: 0.3517
Epoch 2/10
72/72 [==============================] - 0s 6ms/step - loss: 0.3509
Epoch 3/10
72/72 [==============================] - 0s 6ms/step - loss: 0.3509
Epoch 4/10
72/72 [==============================] - 0s 6ms/step - loss: 0.3509
Epoch 5/10
72/72 [==============================] - 0s 6ms/step - loss: 0.3509
Epoch 6/10
72/72 [==============================] - 0s 6ms/step - loss: 0.3509
Epoch 7/10
72/72 [==============================] - 0s 6ms/step - loss: 0.3509
Epoch 8/10
72/72 [==============================] - 0s 6ms/step - loss: 0.3509
Epoch 9/10
72/72 [==============================] - 0s 6ms/step - loss: 0.3509
Epoch 10/10
72/72 [==============================] - 0s 6ms/step - loss: 0.3508


In [47]:
X_pred = autoencoder.predict(X_test_dur)

from sklearn.metrics import r2_score

print(r2_score(X_test_dur, X_pred))

13/13 [==============================] - 0s 4ms/step
-4.594307993125025


In [48]:
enc_model = Model(input_img, encoder)
X_enc = enc_model.predict(X_dur)
X_test_enc = enc_model.predict(X_test_dur)

13/13 [==============================] - 0s 2ms/step


In [54]:
X1 = np.concatenate([demo, X_enc], axis=1)
X1_test = np.concatenate([demo_test, X_test_enc], axis=1)

In [58]:
from Models.search_methods import search_cv

search, cv = search_cv(X1, np.ravel(y), 'mlp', 'Classification', 'Grid')

/Users/mvkairov/mambaforge/envs/meco/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mvkairov/mambaforge/envs/meco/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mvkairov/mambaforge/envs/meco/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mvkairov/mambaforge/envs/meco/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged ye

KeyboardInterrupt: 